# SQL Summarization with SQLite + Groq

This lab is a summarization demo using SQLite for storage and Groq for the LLM call.

**Learning objectives**
- Store text data in SQLite
- Query with SQL
- Summarize with an LLM and store results
- Review results with SQL


In [ ]:
# Optional: install deps if running in a fresh environment
# !pip install groq python-dotenv

In [ ]:
import os
import sqlite3
from dotenv import load_dotenv
from groq import Groq

load_dotenv()
client = Groq(api_key=os.getenv('GROQ_API_KEY'))

In [ ]:
conn = sqlite3.connect('support_tickets.db')
cursor = conn.cursor()

cursor.execute('''
    CREATE TABLE IF NOT EXISTS tickets (
        ticket_id INTEGER PRIMARY KEY,
        customer_name TEXT,
        category TEXT,
        transcript TEXT,
        summary TEXT
    )
''')

sample_tickets = [
    (1001, 'Alice Brown', 'Delivery', 'The product was delayed by three days and I missed a deadline.'),
    (1002, 'Rajesh Kumar', 'Payment', 'Payment failed but money was deducted; I need a refund.'),
    (1003, 'Maria Lopez', 'Account', 'I cannot reset my password and the link keeps expiring.'),
    (1004, 'Jun Park', 'Product', 'The device overheats after 10 minutes of use.'),
]

cursor.executemany(
    'INSERT OR IGNORE INTO tickets (ticket_id, customer_name, category, transcript) VALUES (?, ?, ?, ?)',
    sample_tickets
)
conn.commit()

In [ ]:
# SQL query: find tickets missing summaries
cursor.execute('SELECT ticket_id, transcript FROM tickets WHERE summary IS NULL')
tickets_to_process = cursor.fetchall()
tickets_to_process

In [ ]:
def summarize(text):
    response = client.chat.completions.create(
        model='llama-3.3-70b-versatileo-mini',
        messages=[
            {'role': 'system', 'content': 'Summarize this support ticket in one sentence.'},
            {'role': 'user', 'content': text}
        ]
    )
    return response.choices[0].message.content.strip()

In [ ]:
for ticket_id, transcript in tickets_to_process:
    summary = summarize(transcript)
    cursor.execute('UPDATE tickets SET summary = ? WHERE ticket_id = ?', (summary, ticket_id))

conn.commit()

In [ ]:
cursor.execute('SELECT ticket_id, customer_name, category, summary FROM tickets')
cursor.fetchall()